In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
!pip install spafe
!pip install praat-parselmouth
!pip install textstat
!pip install pocketsphinx
!pip install ctranslate2==4.4.0

In [ ]:
!pip install git+https://github.com/m-bain/whisperx.git


In [ ]:
import os
import pandas as pd

os.chdir("/content/gdrive/MyDrive/speech_analysis")

In [ ]:
from process_file import process_file, process_file_model

In [ ]:
import whisperx
import gc
import torch

torch.set_num_threads(1)

vad_model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

device = "cuda"
batch_size = 8 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
transcription_model = whisperx.load_model("large-v3", device, compute_type=compute_type)

In [ ]:
def add_group_names(output_file):
    names = ["Pausing behavior", "Speech behavior", "Frequency Parameters", "Spectral Domain", "Voice Quality",
             "Loudness and Intensity", "Complexity", "Info"]

    ranges = [(0, 11), (11, 137),(137, 377), (377, 6331), (6331, 6475), (6475, 6715), (6715, 6859), (6859, 6861)]

    group_names = [""] * 6861
    for i, r in enumerate(ranges):
        for j in range(r[0], r[1]):
            group_names[j] = names[i]
    train_df = pd.read_csv(output_file)

    column_names = zip(group_names, train_df.columns)
    train_df.columns = pd.MultiIndex.from_tuples(column_names)

    train_df.to_csv(output_file, index=False)

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import csv


def calculate_all_features(input_dir, output_file, label):
    previously_calculated = {}
    if os.path.exists(output_file):
        features_pd = pd.read_csv(output_file)
        previously_calculated = set(features_pd["filename"].tolist())


    csvfile = open(output_file, "a+")
    writer = None
    write_header = True
    if len(previously_calculated) > 0:
        write_header = False
        df = pd.read_csv(output_file)
        fieldnames = list(df.columns)
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    progress_bar = tqdm(range(len(os.listdir(input_dir))), position=0, leave=True)
    for file in os.listdir(input_dir):
        if file in previously_calculated:
            continue
        if file.endswith('.wav'):
                file_path = os.path.join(input_dir, file)
                try:
                    acoustic_features = process_file(file_path)
                    features = process_file_model(file_path, vad_model, utils, transcription_model)
                    features.update(acoustic_features)
                    # features = {}

                except Exception as e:
                    print(e)
                    print(file)
                    features = {}
                features['filename'] = file
                features['label'] = label
                if write_header:
                    writer = csv.DictWriter(csvfile, fieldnames=features.keys())
                    writer.writeheader()
                    write_header = False
                writer.writerows([features])

        progress_bar.update(1)

    csvfile.close()

In [ ]:
directory_path_ad = "/content/gdrive/MyDrive/Data/2021/Audio-denoise_new/test_data_2021/AD-test-denoise"

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
output_file = "out.csv"
calculate_all_features(directory_path_ad, output_file, 1)
add_group_names(output_file)